In [2]:
# imports
import pandas as pd

# Condo Conversion Data Cleaning

**Goal**: Create dataset of condo conversions from 2015-2024 for City of Cambridge

## Testing

A preliminary step towards putting together the cleaned data. The cells in this section are largely for experimentation purposes and can be run to see the effects of various pandas functions

In [3]:
test_years = [2022, 2023]

test_parcel_data = {
    year: pd.read_csv(f"parcel_data/ASSESSING_PropertyDatabase_FY{year}.csv") for year in test_years
}

In [4]:
num_records_by_address_by_year = {year: pd.DataFrame({f"num_records_{year}": parcel["Address"].value_counts()}) for (year, parcel) in test_parcel_data.items()}

In [5]:
# filter for all the items that had 1 parcel before but more than 1 after
df = num_records_by_address_by_year[2022].join(num_records_by_address_by_year[2023])
df.head()

,num_records_2022,num_records_2023
Address,,
8-12 Museum Way,437,437.0
2 Earhart St,230,230.0
75-83 Cambridge Pkwy,206,206.0
303 Third St,193,193.0
10 Rogers St,173,173.0


In [6]:
condos_df = df[(df.num_records_2022 == 1) & (df.num_records_2023 > 1)].reset_index().rename(columns={"index": "Address"})
condos_df.shape

(40, 3)

In [7]:
df_2022 = test_parcel_data[2022].set_index("Address")

In [8]:
# merge data from previous year with current year
merged_data = pd.merge(test_parcel_data[2022], test_parcel_data[2023], how="left", on="Address", suffixes=("_prior", "_post"))
merged_data.head()

,PID_prior,GISID_prior,BldgNum_prior,Address,Unit_prior,StateClassCode_prior,PropertyClass_prior,Zoning_prior,MapLot_prior,LandArea_prior,...,Systems_Plumbing_post,Condition_YearBuilt_post,Condition_InteriorCondition_post,Condition_OverallCondition_post,Condition_OverallGrade_post,Parking_Open_post,Parking_Covered_post,Parking_Garage_post,UnfinishedBasementGross_post,FinishedBasementGross_post
0,7207,112-10,1,18 Ellsworth Ave,NaN,104,TWO-FAM-RES,C-1,112-10,4297,...,NaN,1903.0,Good,Good,Good Very Good,0.0,0.0,NaN,625.0,624.0
1,7208,112-11,1,20 Ellsworth Ave,NaN,199,CONDO-BLDG,C-1,112-11,3600,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,1292.0,0.0
2,7208,112-11,1,20 Ellsworth Ave,NaN,199,CONDO-BLDG,C-1,112-11,3600,...,NaN,1903.0,NaN,Good,Average,0.0,0.0,0.0,0.0,0.0
3,7208,112-11,1,20 Ellsworth Ave,NaN,199,CONDO-BLDG,C-1,112-11,3600,...,NaN,1903.0,NaN,Excellent,Average,0.0,0.0,0.0,0.0,0.0
4,7209,112-11,1,20 Ellsworth Ave,1,102,CONDOMINIUM,NaN,112-11-1,0,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,1292.0,0.0


In [9]:
merged_data.shape

(717384, 121)

In [10]:
# get conversion data
# TODO: we have multiple records per address because there are small differences in the data (i.e. the unit is different / value is different) 
#       ==> do we want to keep those or remove? 
conversion_data = condos_df.set_index("Address").join(merged_data.set_index("Address"))
conversion_data.head()

,num_records_2022,num_records_2023,PID_prior,GISID_prior,BldgNum_prior,Unit_prior,StateClassCode_prior,PropertyClass_prior,Zoning_prior,MapLot_prior,...,Systems_Plumbing_post,Condition_YearBuilt_post,Condition_InteriorCondition_post,Condition_OverallCondition_post,Condition_OverallGrade_post,Parking_Open_post,Parking_Covered_post,Parking_Garage_post,UnfinishedBasementGross_post,FinishedBasementGross_post
Address,,,,,,,,,,,,,,,,,,,,,
100 Banks St,1,13.0,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,...,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0
100 Banks St,1,13.0,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,...,NaN,1902.0,NaN,Good,Good,1.0,0.0,0.0,0.0,0.0
100 Banks St,1,13.0,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,...,NaN,10902.0,NaN,Good,Good,1.0,0.0,0.0,0.0,0.0
100 Banks St,1,13.0,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,...,NaN,1902.0,NaN,Good,Good,1.0,0.0,0.0,0.0,0.0
100 Banks St,1,13.0,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,...,NaN,1902.0,NaN,Good,Good,1.0,0.0,0.0,0.0,0.0


In [11]:
conversion_data = conversion_data.rename(columns={"num_records_2023": "num_condo_units"}).drop(columns=["num_records_2022"])
conversion_data["num_condo_units"] = conversion_data["num_condo_units"].astype(int)
conversion_data["prior_year"] = 2022
conversion_data["post_year"] = 2023
conversion_data.head()

,num_condo_units,PID_prior,GISID_prior,BldgNum_prior,Unit_prior,StateClassCode_prior,PropertyClass_prior,Zoning_prior,MapLot_prior,LandArea_prior,...,Condition_InteriorCondition_post,Condition_OverallCondition_post,Condition_OverallGrade_post,Parking_Open_post,Parking_Covered_post,Parking_Garage_post,UnfinishedBasementGross_post,FinishedBasementGross_post,prior_year,post_year
Address,,,,,,,,,,,,,,,,,,,,,
100 Banks St,13,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,13451,...,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,2022,2023
100 Banks St,13,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,13451,...,NaN,Good,Good,1.0,0.0,0.0,0.0,0.0,2022,2023
100 Banks St,13,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,13451,...,NaN,Good,Good,1.0,0.0,0.0,0.0,0.0,2022,2023
100 Banks St,13,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,13451,...,NaN,Good,Good,1.0,0.0,0.0,0.0,0.0,2022,2023
100 Banks St,13,10217,131-47,1,NaN,112,>8-UNIT-APT,C-1,131-47,13451,...,NaN,Good,Good,1.0,0.0,0.0,0.0,0.0,2022,2023


## Data Wrangling

This set of cells actually does the data wrangling to put it into the format that we desire. 

In [48]:
years = [i for i in range(2015, 2025)]
parcel_data = {year: pd.read_csv(f"parcel_data/ASSESSING_PropertyDatabase_FY{year}.csv") for year in years}

In [49]:
# # uncomment if you'd like to use the 2013 and 2014 data
# # 2013 and 2014 data uses Location as the header instead of Address
# parcel_data[2013] = parcel_data[2013].rename(columns={"Location": "Address"})
# parcel_data[2014] = parcel_data[2014].rename(columns={"Location": "Address"})

In [50]:
# make all upper to account for different capitalization schemes
for year in years:
    parcel_data[year]['Address'] = parcel_data[year]['Address'].str.upper()

In [51]:
parcel_data[2015]["GISID"] = parcel_data[2015]["GISID"].apply(lambda id: id[2:-1])
parcel_data[2015].head()

,PID,GISID,BldgNum,Address,Unit,StateClassCode,PropertyClass,Zoning,Map/Lot,LandArea,...,Systems_Plumbing,Condition_YearBuilt,Condition_InteriorCondition,Condition_OverallCondition,Condition_OverallGrade,Parking_Open,Parking_Covered,Parking_Garage,UnfinishedBasementGross,FinishedBasementGross
0,2,7-18,1,219 MONSIGNOR OBRIEN HWY,NaN,390,COM-DEV-LAND,"=""SD-1""","=""7-18""",12478,...,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0,0.0
1,3,7-19,1,221 MONSIGNOR OBRIEN HWY,NaN,390,COM-DEV-LAND,"=""SD-1""","=""7-19""",11182,...,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0,0.0
2,5,7-29,1,225 MONSIGNOR OBRIEN HWY,NaN,400,MANUFACTURNG,"=""SD-1""","=""7-29""",32681,...,POOR,1920,NaN,Poor,FAIR,NaN,NaN,NaN,0,NaN
3,6,7-31,1,187-197 MONSIGNOR OBRIEN HWY,NaN,300,HOTEL,"=""SD-1""","=""7-31""",12930,...,AVERAGE,2002,NaN,Excellent,EXCELLENT,NaN,NaN,NaN,0,NaN
4,7,7-34,1,199 MONSIGNOR OBRIEN HWY,NaN,316,WAREHOUSE,"=""SD-1""","=""7-34""",10233,...,AVERAGE,1870,NaN,Fair,AVERAGE,NaN,NaN,NaN,0,NaN


In [52]:
num_records_by_address_by_year = {year: pd.DataFrame({f"num_records_{year}": parcel["Address"].value_counts()}) for (year, parcel) in parcel_data.items()}

In [53]:
# get condo conversions between prior_year and prior_year + 1
# returns dataframe with conversion data
def get_condo_conversions_df(prior_year): 
    records_df = num_records_by_address_by_year[prior_year].join(num_records_by_address_by_year[prior_year + 1])

    # condo conversions
    condos_df = records_df[(records_df[f"num_records_{prior_year}"] == 1) & (records_df[f"num_records_{prior_year + 1}"] > 1)]

    # prior dataframe, filtered by addresses that were converted
    prior_df = parcel_data[prior_year].set_index("Address")
    prior_df = prior_df[prior_df.index.isin(condos_df.index)]

    # post dataframe, filtered by addresses that were converted
    post_df = parcel_data[prior_year + 1].set_index("Address")
    post_df = post_df[post_df.index.isin(condos_df.index)]

    merged_df = pd.merge(prior_df, post_df, how="left", on="Address", suffixes=("_prior", "_post"))
    df = (condos_df
          .join(merged_df)
          .reset_index()
          .drop_duplicates(subset="Address")
          .rename(columns={f"num_records_{prior_year + 1}": "num_condo_units"})
          .drop(columns=[f"num_records_{prior_year}"])
          .set_index("Address"))
    df["num_condo_units"] = df.num_condo_units.astype(int)
    df["prior_year"] = prior_year
    df["post_year"] = prior_year + 1
    return df;


In [54]:
all_conversions = pd.DataFrame()
for year in range(2015, 2024): 
    all_conversions = pd.concat([all_conversions, get_condo_conversions_df(year)])

In [55]:
all_conversions.head()

,num_condo_units,PID_prior,GISID_prior,BldgNum_prior,Unit_prior,StateClassCode_prior,PropertyClass_prior,Zoning_prior,Map/Lot_prior,LandArea_prior,...,MapLot,BookPage,PropertyTaxAmount_post,Exterior_Occupancy,MapLot_prior,BookPage_prior,Exterior_Occupancy_prior,MapLot_post,BookPage_post,Exterior_Occupancy_post
Address,,,,,,,,,,,,,,,,,,,,,
10 ELLERY ST,4,8053,116-119,1,NaN,104,TWO-FAM-RES,"=""C-1""","=""116-119""",6082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 AVON PL,3,17759,211-15,1,NaN,104,TWO-FAM-RES,"=""A-2""","=""211-15""",6399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 RINDGEFIELD ST,3,16472,196-91,1,NaN,104,TWO-FAM-RES,"=""B""","=""196-91""",3699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133-135 FRESH POND PKWY,3,21305,259-17,1,NaN,104,TWO-FAM-RES,"=""B""","=""259-17""",4538,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139 CHARLES ST,3,690,17-20,1,NaN,101,SNGL-FAM-RES,"=""C-1""","=""17-20""",2115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Spatial Data

Associate condo conversions to 2024 spatial data

In [32]:
import geopandas as gpd

In [33]:
geo_df = gpd.read_file("./parcel_data/ASSESSING_ParcelsFY2024.geojson")

In [34]:
geo_df.head()

,POLY_TYPE,MAP,UYEAR,LOC_ID,GlobalID,EditDate,ML,SOURCE,created_user,Editor,LOT,created_date,PLAN_ID,last_edited_date,last_edited_user,geometry
0,PRIV_ROW,-,0.0,F_761786_2962533,{1118511F-457E-4275-ACDD-1A0C3D8307F0},None,---,ASSESS,None,None,-,None,None,2023-09-26,SSWEENEY,"POLYGON ((-71.10911 42.37663, -71.10912 42.376..."
1,None,263,0.0,F_754404_2965391,{135EB22F-4680-4A35-8736-A2D1E588AE61},None,263-23,ASSESS,None,None,23,None,None,2023-09-26,SSWEENEY,"POLYGON ((-71.13621 42.38470, -71.13640 42.384..."
2,None,201,0.0,F_758048_2966643,{318E939B-73BB-473A-B3A9-B5E3A6A8AB0B},None,201-95,ASSESS,None,None,95,None,None,2023-09-26,SSWEENEY,"POLYGON ((-71.12275 42.38791, -71.12283 42.387..."
3,None,26,0.0,F_768259_2959811,{728A555A-EACA-48C7-9A75-1E00CD92672D},None,26-37,ASSESS,None,None,37,None,None,2023-09-26,SSWEENEY,"POLYGON ((-71.08510 42.36914, -71.08511 42.369..."
4,None,36,0.0,F_767412_2961233,{7EC94D31-F4E5-4B0C-9AE7-13D946E8A754},None,36-203,ASSESS,None,None,203,None,None,2023-09-26,SSWEENEY,"POLYGON ((-71.08823 42.37308, -71.08825 42.372..."


In [67]:
merged_conversion_df = pd.merge(all_conversions.reset_index(), geo_df, how="left", left_on="GISID_prior", right_on="ML")

In [69]:
merged_conversion_df.to_csv("condo_conversions.csv")